In [1]:
!pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import re
#stopwords
import nltk
from nltk.corpus import stopwords

#stemming
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

#spit data
from sklearn.model_selection import train_test_split

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cd /content/drive/MyDrive/PPW/tugas

/content/drive/MyDrive/PPW/tugas


# **Klasifikasi Berita**

## **Deskripsi Dataset**

Dataset ini berisi berita yang dikelompokkan ke dalam dua kategori, yaitu 'Politik' dan 'Gaya Hidup'.

Jumlah Data : 100 baris data dengan 5 kolom

Dataset ini TIDAK MEMILIKI Missing Values yang berarti bahwa setiap bagian data pada dataset lengkap dengan informasi.

## **Tujuan**

Tujuan dari dataset ini adalah untuk mengklasifikasikan berita ke dalam dua kategori, yaitu "Politik" dan "Gaya Hidup". Dengan menggunakan teknik pemrosesan teks dan model klasifikasi, seperti Logistic Regression, dataset ini bertujuan untuk membangun sistem yang dapat secara otomatis mengidentifikasi kategori berita berdasarkan konten yang diberikan.

## **Penjelasan Fitur**

Jumlah Fitur : 4

* **Judul Berita** : Judul dari setiap berita
* **Isi Berita** : Konten utama dari berita. Kolom ini memuat informasi lengkap tentang peristiwa, fakta, atau opini yang dibahas dalam berita. Fitur ini diproses lebih lanjut untuk analisis teks.
* **Tanggal Berita** : Fitur ini menunjukkan kapan berita tersebut diterbitkan berupa tanggal dan waktu
* **Kategori Berita** : Fitur ini merupakan label atau kategori dari setiap berita. Kategori ini menunjukkan apakah berita tersebut berkaitan dengan Politik atau Gaya Hidup. Fitur ini penting karena merupakan target yang akan diprediksi dalam model klasifikasi. Kolom kategori ini nantinya akan diubah menjadi nilai numerik (misalnya, 0 untuk Politik dan 1 untuk Gaya Hidup).

## **Sumber Dataset**

[https://timesindonesia.co.id/#google_vignette](https://timesindonesia.co.id/#google_vignette)

Dataset ini berisi kumpulan berita yang diambil dari situs TIMES Indonesia, sebuah portal berita yang memuat berbagai artikel terkait berbagai topik, termasuk politik dan gaya hidup. Data ini dikumpulkan melalui proses web crawling menggunakan Python, yang merupakan teknik pengumpulan data secara otomatis dari situs web.

## **Dataset**

In [5]:
df = pd.read_csv("Tugas-Crawling-Data-Berita-2-kategori.csv")

df

,NO,Judul Berita,Isi Berita,Tanggal Berita,Kategori Berita
0,1,"Analisis PPI: Di Tangan Presiden Prabowo, Indo...","TIMESINDONESIA, JAKARTA – Di bawah kepemimpina...",04/09/2024 - 23:35,Politik
1,2,"Bacawabup Lathifah Solidkan Seribu Relawan, Si...","TIMESINDONESIA, MALANG – Bakal cawabup Malang,...",04/09/2024 - 22:02,Politik
2,3,Rois Syuriah Sebut Visi Misi Untoro-Wahyudi Se...,"TIMESINDONESIA, BANTUL – Rois Syuriah PCNU Ban...",04/09/2024 - 20:54,Politik
3,4,Ribuan Kader PKB Kabupaten Mojokerto Panasi Me...,"TIMESINDONESIA, MOJOKERTO – Ribuan kader PKB K...",04/09/2024 - 19:23,Politik
4,5,"Wajah Pengurus Inti DPP PKB Tidak Berubah, Ini...","TIMESINDONESIA, JAKARTA – Partai Kebangkitan B...",04/09/2024 - 16:43,Politik
...,...,...,...,...,...
95,96,Azquira Scarf Luncurkan Hijab Autentik dengan ...,"TIMESINDONESIA, MALANG – Azquira Scarf menggel...",01/09/2024 - 22:42,Gaya Hidup
96,97,Mengulik Sejarah Bangunan Heritage Kota Malang...,"TIMESINDONESIA, MALANG – Pada Minggu (1/9/2024...",01/09/2024 - 22:16,Gaya Hidup
97,98,"Ritual Wewangian, Parfum Lokal Jadi Pilihan In...","TIMESINDONESIA, SURABAYA – Memilih wewangian p...",31/08/2024 - 20:04,Gaya Hidup
98,99,Berkeliling Nusantara Melalui Suguhan Kuliner ...,"TIMESINDONESIA, MAGELANG – Anda ingin mencicip...",04/09/2024 - 17:00,Gaya Hidup


## **Preprocessing**

### **Label Encoder**

In [10]:
df['Kategori'] = df['Kategori Berita'].map({'Politik': 0, 'Gaya Hidup': 1})

### **Cleaning**

In [11]:
def remove_symbols(data_berita):
  data_berita = re.sub(r'[^a-zA-Z0-9\s]', '', data_berita) #menghapus karakter
  return data_berita

df['cleansing'] = df['Isi Berita'].apply(lambda x: remove_symbols(x))

df[['Isi Berita', 'cleansing']]

,Isi Berita,cleansing
0,"TIMESINDONESIA, JAKARTA – Di bawah kepemimpina...",TIMESINDONESIA JAKARTA Di bawah kepemimpinan ...
1,"TIMESINDONESIA, MALANG – Bakal cawabup Malang,...",TIMESINDONESIA MALANG Bakal cawabup Malang La...
2,"TIMESINDONESIA, BANTUL – Rois Syuriah PCNU Ban...",TIMESINDONESIA BANTUL Rois Syuriah PCNU Bantu...
3,"TIMESINDONESIA, MOJOKERTO – Ribuan kader PKB K...",TIMESINDONESIA MOJOKERTO Ribuan kader PKB Kab...
4,"TIMESINDONESIA, JAKARTA – Partai Kebangkitan B...",TIMESINDONESIA JAKARTA Partai Kebangkitan Ban...
...,...,...
95,"TIMESINDONESIA, MALANG – Azquira Scarf menggel...",TIMESINDONESIA MALANG Azquira Scarf menggelar...
96,"TIMESINDONESIA, MALANG – Pada Minggu (1/9/2024...",TIMESINDONESIA MALANG Pada Minggu 192024 komu...
97,"TIMESINDONESIA, SURABAYA – Memilih wewangian p...",TIMESINDONESIA SURABAYA Memilih wewangian pen...
98,"TIMESINDONESIA, MAGELANG – Anda ingin mencicip...",TIMESINDONESIA MAGELANG Anda ingin mencicipi ...


### **Case Folding**

In [12]:
def case_folding(text):
    if isinstance(text, str):
      lowercase_text = text.lower()
      return lowercase_text
    else :
      return text

df ['case_folding'] = df['cleansing'].apply(case_folding)

df[['Isi Berita', 'cleansing', 'case_folding']]

,Isi Berita,cleansing,case_folding
0,"TIMESINDONESIA, JAKARTA – Di bawah kepemimpina...",TIMESINDONESIA JAKARTA Di bawah kepemimpinan ...,timesindonesia jakarta di bawah kepemimpinan ...
1,"TIMESINDONESIA, MALANG – Bakal cawabup Malang,...",TIMESINDONESIA MALANG Bakal cawabup Malang La...,timesindonesia malang bakal cawabup malang la...
2,"TIMESINDONESIA, BANTUL – Rois Syuriah PCNU Ban...",TIMESINDONESIA BANTUL Rois Syuriah PCNU Bantu...,timesindonesia bantul rois syuriah pcnu bantu...
3,"TIMESINDONESIA, MOJOKERTO – Ribuan kader PKB K...",TIMESINDONESIA MOJOKERTO Ribuan kader PKB Kab...,timesindonesia mojokerto ribuan kader pkb kab...
4,"TIMESINDONESIA, JAKARTA – Partai Kebangkitan B...",TIMESINDONESIA JAKARTA Partai Kebangkitan Ban...,timesindonesia jakarta partai kebangkitan ban...
...,...,...,...
95,"TIMESINDONESIA, MALANG – Azquira Scarf menggel...",TIMESINDONESIA MALANG Azquira Scarf menggelar...,timesindonesia malang azquira scarf menggelar...
96,"TIMESINDONESIA, MALANG – Pada Minggu (1/9/2024...",TIMESINDONESIA MALANG Pada Minggu 192024 komu...,timesindonesia malang pada minggu 192024 komu...
97,"TIMESINDONESIA, SURABAYA – Memilih wewangian p...",TIMESINDONESIA SURABAYA Memilih wewangian pen...,timesindonesia surabaya memilih wewangian pen...
98,"TIMESINDONESIA, MAGELANG – Anda ingin mencicip...",TIMESINDONESIA MAGELANG Anda ingin mencicipi ...,timesindonesia magelang anda ingin mencicipi ...


### **Tokenize**

In [13]:
def tokenize(text):
    tokens = text.split()
    return tokens

df['tokenize'] = df['case_folding'].apply(tokenize)

df[['Isi Berita', 'cleansing', 'case_folding', 'tokenize']]

,Isi Berita,cleansing,case_folding,tokenize
0,"TIMESINDONESIA, JAKARTA – Di bawah kepemimpina...",TIMESINDONESIA JAKARTA Di bawah kepemimpinan ...,timesindonesia jakarta di bawah kepemimpinan ...,"[timesindonesia, jakarta, di, bawah, kepemimpi..."
1,"TIMESINDONESIA, MALANG – Bakal cawabup Malang,...",TIMESINDONESIA MALANG Bakal cawabup Malang La...,timesindonesia malang bakal cawabup malang la...,"[timesindonesia, malang, bakal, cawabup, malan..."
2,"TIMESINDONESIA, BANTUL – Rois Syuriah PCNU Ban...",TIMESINDONESIA BANTUL Rois Syuriah PCNU Bantu...,timesindonesia bantul rois syuriah pcnu bantu...,"[timesindonesia, bantul, rois, syuriah, pcnu, ..."
3,"TIMESINDONESIA, MOJOKERTO – Ribuan kader PKB K...",TIMESINDONESIA MOJOKERTO Ribuan kader PKB Kab...,timesindonesia mojokerto ribuan kader pkb kab...,"[timesindonesia, mojokerto, ribuan, kader, pkb..."
4,"TIMESINDONESIA, JAKARTA – Partai Kebangkitan B...",TIMESINDONESIA JAKARTA Partai Kebangkitan Ban...,timesindonesia jakarta partai kebangkitan ban...,"[timesindonesia, jakarta, partai, kebangkitan,..."
...,...,...,...,...
95,"TIMESINDONESIA, MALANG – Azquira Scarf menggel...",TIMESINDONESIA MALANG Azquira Scarf menggelar...,timesindonesia malang azquira scarf menggelar...,"[timesindonesia, malang, azquira, scarf, mengg..."
96,"TIMESINDONESIA, MALANG – Pada Minggu (1/9/2024...",TIMESINDONESIA MALANG Pada Minggu 192024 komu...,timesindonesia malang pada minggu 192024 komu...,"[timesindonesia, malang, pada, minggu, 192024,..."
97,"TIMESINDONESIA, SURABAYA – Memilih wewangian p...",TIMESINDONESIA SURABAYA Memilih wewangian pen...,timesindonesia surabaya memilih wewangian pen...,"[timesindonesia, surabaya, memilih, wewangian,..."
98,"TIMESINDONESIA, MAGELANG – Anda ingin mencicip...",TIMESINDONESIA MAGELANG Anda ingin mencicipi ...,timesindonesia magelang anda ingin mencicipi ...,"[timesindonesia, magelang, anda, ingin, mencic..."


### **Stop Word Removal**

In [14]:
nltk.download('stopwords')
stop_words = stopwords.words('indonesian')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [15]:
def remove_stopwords(text):
  return [word for word in text if word not in stop_words]

df['stopword_removal'] = df['tokenize'].apply(lambda x: ' '.join(remove_stopwords(x)))

df[['Isi Berita', 'cleansing', 'case_folding', 'tokenize', 'stopword_removal']]

,Isi Berita,cleansing,case_folding,tokenize,stopword_removal
0,"TIMESINDONESIA, JAKARTA – Di bawah kepemimpina...",TIMESINDONESIA JAKARTA Di bawah kepemimpinan ...,timesindonesia jakarta di bawah kepemimpinan ...,"[timesindonesia, jakarta, di, bawah, kepemimpi...",timesindonesia jakarta kepemimpinan presiden r...
1,"TIMESINDONESIA, MALANG – Bakal cawabup Malang,...",TIMESINDONESIA MALANG Bakal cawabup Malang La...,timesindonesia malang bakal cawabup malang la...,"[timesindonesia, malang, bakal, cawabup, malan...",timesindonesia malang cawabup malang lathifah ...
2,"TIMESINDONESIA, BANTUL – Rois Syuriah PCNU Ban...",TIMESINDONESIA BANTUL Rois Syuriah PCNU Bantu...,timesindonesia bantul rois syuriah pcnu bantu...,"[timesindonesia, bantul, rois, syuriah, pcnu, ...",timesindonesia bantul rois syuriah pcnu bantul...
3,"TIMESINDONESIA, MOJOKERTO – Ribuan kader PKB K...",TIMESINDONESIA MOJOKERTO Ribuan kader PKB Kab...,timesindonesia mojokerto ribuan kader pkb kab...,"[timesindonesia, mojokerto, ribuan, kader, pkb...",timesindonesia mojokerto ribuan kader pkb kabu...
4,"TIMESINDONESIA, JAKARTA – Partai Kebangkitan B...",TIMESINDONESIA JAKARTA Partai Kebangkitan Ban...,timesindonesia jakarta partai kebangkitan ban...,"[timesindonesia, jakarta, partai, kebangkitan,...",timesindonesia jakarta partai kebangkitan bang...
...,...,...,...,...,...
95,"TIMESINDONESIA, MALANG – Azquira Scarf menggel...",TIMESINDONESIA MALANG Azquira Scarf menggelar...,timesindonesia malang azquira scarf menggelar...,"[timesindonesia, malang, azquira, scarf, mengg...",timesindonesia malang azquira scarf menggelar ...
96,"TIMESINDONESIA, MALANG – Pada Minggu (1/9/2024...",TIMESINDONESIA MALANG Pada Minggu 192024 komu...,timesindonesia malang pada minggu 192024 komu...,"[timesindonesia, malang, pada, minggu, 192024,...",timesindonesia malang minggu 192024 komunitas ...
97,"TIMESINDONESIA, SURABAYA – Memilih wewangian p...",TIMESINDONESIA SURABAYA Memilih wewangian pen...,timesindonesia surabaya memilih wewangian pen...,"[timesindonesia, surabaya, memilih, wewangian,...",timesindonesia surabaya memilih wewangian peng...
98,"TIMESINDONESIA, MAGELANG – Anda ingin mencicip...",TIMESINDONESIA MAGELANG Anda ingin mencicipi ...,timesindonesia magelang anda ingin mencicipi ...,"[timesindonesia, magelang, anda, ingin, mencic...",timesindonesia magelang mencicipi kenikmatan s...


### **Stemming**

In [16]:
# Inisialisasi stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Fungsi stemming
def stemming(text):
    return [stemmer.stem(word) for word in text]

# Terapkan stemming setelah stopwords removal
df['stemming'] = df['stopword_removal'].apply(lambda x: ' '.join(stemming(x.split())))

df[['Isi Berita', 'cleansing', 'case_folding', 'tokenize', 'stopword_removal', 'stemming']]

,Isi Berita,cleansing,case_folding,tokenize,stopword_removal,stemming
0,"TIMESINDONESIA, JAKARTA – Di bawah kepemimpina...",TIMESINDONESIA JAKARTA Di bawah kepemimpinan ...,timesindonesia jakarta di bawah kepemimpinan ...,"[timesindonesia, jakarta, di, bawah, kepemimpi...",timesindonesia jakarta kepemimpinan presiden r...,timesindonesia jakarta pimpin presiden ri prab...
1,"TIMESINDONESIA, MALANG – Bakal cawabup Malang,...",TIMESINDONESIA MALANG Bakal cawabup Malang La...,timesindonesia malang bakal cawabup malang la...,"[timesindonesia, malang, bakal, cawabup, malan...",timesindonesia malang cawabup malang lathifah ...,timesindonesia malang cawabup malang lathifah ...
2,"TIMESINDONESIA, BANTUL – Rois Syuriah PCNU Ban...",TIMESINDONESIA BANTUL Rois Syuriah PCNU Bantu...,timesindonesia bantul rois syuriah pcnu bantu...,"[timesindonesia, bantul, rois, syuriah, pcnu, ...",timesindonesia bantul rois syuriah pcnu bantul...,timesindonesia bantul rois syuriah pcnu bantul...
3,"TIMESINDONESIA, MOJOKERTO – Ribuan kader PKB K...",TIMESINDONESIA MOJOKERTO Ribuan kader PKB Kab...,timesindonesia mojokerto ribuan kader pkb kab...,"[timesindonesia, mojokerto, ribuan, kader, pkb...",timesindonesia mojokerto ribuan kader pkb kabu...,timesindonesia mojokerto ribu kader pkb kabupa...
4,"TIMESINDONESIA, JAKARTA – Partai Kebangkitan B...",TIMESINDONESIA JAKARTA Partai Kebangkitan Ban...,timesindonesia jakarta partai kebangkitan ban...,"[timesindonesia, jakarta, partai, kebangkitan,...",timesindonesia jakarta partai kebangkitan bang...,timesindonesia jakarta partai bangkit bangsa p...
...,...,...,...,...,...,...
95,"TIMESINDONESIA, MALANG – Azquira Scarf menggel...",TIMESINDONESIA MALANG Azquira Scarf menggelar...,timesindonesia malang azquira scarf menggelar...,"[timesindonesia, malang, azquira, scarf, mengg...",timesindonesia malang azquira scarf menggelar ...,timesindonesia malang azquira scarf gelar acar...
96,"TIMESINDONESIA, MALANG – Pada Minggu (1/9/2024...",TIMESINDONESIA MALANG Pada Minggu 192024 komu...,timesindonesia malang pada minggu 192024 komu...,"[timesindonesia, malang, pada, minggu, 192024,...",timesindonesia malang minggu 192024 komunitas ...,timesindonesia malang minggu 192024 komunitas ...
97,"TIMESINDONESIA, SURABAYA – Memilih wewangian p...",TIMESINDONESIA SURABAYA Memilih wewangian pen...,timesindonesia surabaya memilih wewangian pen...,"[timesindonesia, surabaya, memilih, wewangian,...",timesindonesia surabaya memilih wewangian peng...,timesindonesia surabaya pilih wewangian harum ...
98,"TIMESINDONESIA, MAGELANG – Anda ingin mencicip...",TIMESINDONESIA MAGELANG Anda ingin mencicipi ...,timesindonesia magelang anda ingin mencicipi ...,"[timesindonesia, magelang, anda, ingin, mencic...",timesindonesia magelang mencicipi kenikmatan s...,timesindonesia magelang cicip nikmat deret kul...


In [17]:
# Menyimpan data ke dalam file CSV
df.to_csv('/content/drive/MyDrive/PPW/tugas/data-preprosesing(klasifikasiberita).csv', index=False)

### **Split Data**

In [18]:
x = df['stemming'].values
y = df['Kategori'].values

In [19]:
Xtrain, Xtest,Ytrain,Ytest = train_test_split(x,y,test_size=0.2,random_state=2)

In [20]:
# Mengonversi ke DataFrame
df_train = pd.DataFrame({'Isi Berita': Xtrain, 'Kategori': Ytrain})
df_test = pd.DataFrame({'Isi Berita': Xtest, 'Kategori': Ytest})

In [21]:
### Data Training
df_train

,Isi Berita,Kategori
0,timesindonesia mojokerto ribu kader pkb kabupa...,0
1,timesindonesia mojokerto ribu kader pkb kabupa...,0
2,timesindonesia jakarta pepaya tanam asli indon...,1
3,timesindonesia jakarta partai bangkit bangsa p...,0
4,timesindonesia surabaya pilih wewangian harum ...,1
...,...,...
75,timesindonesia mojokerto ribu kader pkb kabupa...,0
76,timesindonesia jakarta rabu 492024 hiruk pikuk...,0
77,timesindonesia malang minggu 192024 komunitas ...,1
78,timesindonesia surabaya pasang khofifah indar ...,0


In [22]:
### Data Testing
df_test

,Isi Berita,Kategori
0,timesindonesia jakarta tauco salah bumbu tradi...,1
1,timesindonesia jakarta rabu 492024 hiruk pikuk...,0
2,timesindonesia malang minggu 192024 komunitas ...,1
3,timesindonesia jakarta pimpin presiden ri prab...,0
4,timesindonesia jakarta pimpin presiden ri prab...,0
5,timesindonesia surabaya pasang khofifah indar ...,0
6,timesindonesia bantul rois syuriah pcnu bantul...,0
7,timesindonesia mojokerto ribu kader pkb kabupa...,0
8,timesindonesia jakarta partai bangkit bangsa p...,0
9,timesindonesia lombok timur bupati lombok timu...,0


## **TF-IDF**

In [23]:
# Inisialisasi TfidfVectorizer
vect = TfidfVectorizer()

# Fitting dan transformasi data pelatihan (mengajarkan model tentang kata-kata di Xtrain dan menghitung TF-IDF)
X = vect.fit_transform(Xtrain)

# Mengonversi Xtrain menjadi representasi TF-IDF
X_array = vect.transform(Xtrain)

In [24]:
# Mengubah matriks menjadi array
X_array.toarray()

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.08404587],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.08404587],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [26]:
X_array.shape

(80, 1584)

In [27]:
# Menyimpan dalam pkl
with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(vect, f)

## **Logistic Regression**

In [28]:
# Membuat model Logistic Regression
model = LogisticRegression()

# Melatih model dengan data training
model.fit(X, Ytrain)

LogisticRegression()

In [30]:
# Meyimpan model
with open('logistic_model.pkl', 'wb') as f:
    pickle.dump(model, f)

## **Implementasi menggunakan data baru (Input User)**

### Load TFIDF dan Model LR data test

In [31]:
# Load TF-IDF dan model logistic regression
with open('tfidf_vectorizer.pkl', 'rb') as f:
    tfidf_vectorizer = pickle.load(f)

with open('logistic_model.pkl', 'rb') as f:
    logistic_model = pickle.load(f)

In [34]:
# Transformasi data baru dengan TF-IDF yang sudah dilatih
X_new = tfidf_vectorizer.transform(Xtest)

# Lakukan prediksi dengan model logistic regression
prediksi = model.predict(X_new)

In [35]:
accuracy = accuracy_score(Ytest, prediksi)
print(f"Akurasi: {accuracy * 100:.2f}%")

Akurasi: 100.00%


### Preprosesing

In [ ]:
# memPreprosesing input pengguna
def preprocess_input(text):
    text = remove_symbols(text)
    text = case_folding(text)
    tokens = tokenize(text)
    tokens = remove_stopwords(tokens)
    tokens = stemming(tokens)
    return ' '.join(tokens)

### Klasifikasi Kategori

In [ ]:
# Untuk mengklasifikasi kategori dari input pengguna
def predict_category(user_input):
    # Preprocessing input
    preprocessed_input = preprocess_input(user_input)

    # Transformasi input pengguna menggunakan TF-IDF
    input_tfidf = tfidf_vectorizer.transform([preprocessed_input])

    # mengklasifikasi Kategori menggunakan MODEL LOGISTIC REGRESSION
    prediction = logistic_model.predict(input_tfidf)

    # Memetakan presiksi kategori
    category_map = {0: 'Politik', 1: 'Gaya Hidup'}
    return category_map[prediction[0]]

### Input Pengguna

In [ ]:
# input pengguna
user_input = input("Masukkan isi berita: ")
predicted_category = predict_category(user_input)
print(f"Kategori berita: {predicted_category}")

Masukkan isi berita: TIMESINDONESIA, JEMBER – Sebanyak 142 barista dari berbagai daerah mengikuti Pandhalungan Coffee Week 2024 yang berlangsung di Main Atrium GF Lippo Plaza Jember, Jember, Jawa Timur (Jatim). Pandhalungan Coffee Week yang pertama kali digelar pada 2023 tersebut menjadi ajang berbagi cerita dan promosi bagi pecinta kopi Nusantara serta pelaku usaha yang tidak jauh dari biji kopi. "Jadi pada awalnya berdiri kegiatan ini di tahun 2023 itu hanya segmented untuk kopi. Kemudian pada tahun ini kami menginisiasi bahwa spektrumnya kami perluas," kata Resty Subagio selaku Ketua Panitia Pandhalungan Coffee Week saat ditemui di Lippo Plaza Jember, Sabtu (12/10/2024).  Resty mengatakan, Jember merupakan kabupaten yang dikenal sebagai salah satu penghasil kopi terbaik di Indonesia, khususnya kopi robusta.  Namun, dia menyatakan bahwa perhatian terhadap potensi kopi di Jember masih kurang.  "Harapannya setelah ada klaim seperti itu, kami ingin punya banyak acara yang dapat mengangk

### [Deploy Streamlit ](https://berita-ppw.streamlit.app/)